In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd

algorithm="algorithm";benchmark="bench_prior";seed="seed";time="used_fidelity";loss="value"
data=pd.read_parquet("priorband_benchPrior_relRanks_f24_meta.parquet")
algos=["pb_mutation_dynamic_geometric-default-at-target","random_search","hyperband"]
benchmarks=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]
data_f24=data.loc[data["used_fidelity"]==24.0]
label_dict={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}
bench_priors=data[benchmark].unique().tolist()

r_data=sm.datasets.get_rdataset("dietox", "geepack").data
md = smf.mixedlm("Weight ~ Time", r_data, groups=r_data["Pig"])
mdf=md.fit(method=["lbfgs"])
def convert_to_int(row):
    return pd.Series([bench_priors.index(row[benchmark]),list(label_dict.values()).index(row[algorithm])])

#data[[benchmark,algorithm]]=data.apply(convert_to_int,axis=1).astype(int)
data[[benchmark,algorithm]]=data[[benchmark,algorithm]]

In [2]:
mo = smf.mixedlm("value~C(algorithm)", data, groups=benchmark,re_formula="0",vc_formula={benchmark:f'0+C({benchmark})'})
mof=mo.fit(method=["lbfgs"])
print(mof.summary())


c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1534: RuntimeWarning: divide by zero encountered in log
  cov_aug_logdet = cov_re_logdet + np.sum(np.log(vc_var))
c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1538: RuntimeWarning: divide by zero encountered in divide
  solver = _smw_solver(1., ex_r, ex2_r, cov_re_inv, 1 / vc_var)
c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\regression\mixed_linear_model.py:1543: RuntimeWarning: divide by zero encountered in divide
  ld = _smw_logdet(1., ex_r, ex2_r, cov_re_inv, 1 / vc_var,
c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\regression\mixed_linear_model.py:573: RuntimeWarning: invalid value encountered in scalar add
  return B_logdet + ld + ld1
c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\re

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  value       
No. Observations:   86400    Method:              REML        
No. Groups:         24       Scale:               16.5230     
Min. group size:    3600     Log-Likelihood:      -243894.5917
Max. group size:    3600     Converged:           Yes         
Mean group size:    3600.0                                    
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            2.263    3.319  0.682 0.495 -4.242  8.769
C(algorithm)[T.PB]   0.293    0.034  8.660 0.000  0.227  0.360
C(algorithm)[T.RS]  -0.100    0.034 -2.953 0.003 -0.166 -0.034
bench_prior Var    264.368                                    



c:\Users\Amega\Git\significance_analysis\.venv_3_10_0\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
